<a href="https://colab.research.google.com/github/KenYew/sre-notes/blob/main/linux_internals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 💻 Linux Internals

## 📖 Terminology
1. **Operating system:** This refers to the software and files that are installed on a system so
that it can boot and execute programs. It includes the kernel, administration tools, and
system libraries.
1. **Kernel:** The kernel is the program that manages the system, including (depending on the
kernel model) hardware devices, memory, and CPU scheduling. It runs in a privileged
CPU mode that allows direct access to hardware, called kernel mode.
1. **Process:** An OS abstraction and environment for executing a program. The program runs
in user mode, with access to kernel mode (e.g., for performing device I/O) via system calls
or traps into the kernel.
1. **Thread:** An executable context that can be scheduled to run on a CPU. The kernel has
multiple threads, and a process contains one or more.
1. **Task:** A Linux runnable entity, which can refer to a process (with a single thread), a thread
from a multithreaded process, or kernel threads.
1. **BPF program:** A kernel-mode program running in the BPF1 execution environment.
1. **Main memory:** The physical memory of the system (e.g., RAM).
1. **Virtual memory:** An abstraction of main memory that supports multitasking and oversubscription.
It is, practically, an infinite resource.
1. **Kernel space:** The virtual memory address space for the kernel.
1. **User space:** The virtual memory address space for processes.
1. **User land:** User-level programs and libraries (e.g.: /usr/bin, /usr/lib).
1. **Context switch:** A switch from running one thread or process to another. This is a normal
function of the kernel CPU scheduler, and involves switching the set of running CPU
registers (the thread context) to a new set.
1. **Mode switch:** A switch between kernel and user modes.
1. **System call (syscall):** A well-defined protocol for user programs to request the kernel to
perform privileged operations, including device I/O.
1. **Processor:** Not to be confused with process, a processor is a physical chip containing one
or more CPUs.
1. **Trap:** A signal sent to the kernel to request a system routine (privileged action). Trap types
include system calls, processor exceptions, and interrupts.
1. **Hardware interrupt:** A signal sent by physical devices to the kernel, usually to request
servicing of I/O. An interrupt is a type of trap.

## 👑 Linux Filesystem Hierachy Standard (FHS)

<img src="https://drive.google.com/uc?id=19SBbf-eijcZqvPqigYUKV_A5iNnUC-1q" align="middle" width="500px"/>

| No. | Directory	| Description |
|:--|:--|:--|
| 1. | bin | Core operating system commands | 
| 2. | /boot | Boot loader, kernel, and files needed by the kernel
| 3. | /compat | On FreeBSD, files and libraries for Linux binary compatibility
| 4. | /dev | Device entries for disks, printers, pseudo-terminals, etc.
| 5. | /etc | Critical startup and configuration files
| 6. | /home | Default home directories for users
| 7. | /lib | Libraries, shared libraries, and commands used by /bin and /sbin
| 8. | /media | Mount points for filesystems on removable media
| 9. | /mnt | Temporary mount points, mounts for removable media
| 10. | /opt | Optional software packages (rarely used, for compatibility)
| 11. | /proc | Information about all running processes
| 12. | /root | Home directory of the superuser (sometimes just /)
| 13. | /run | Rendezvous points for running programs (PIDs, sockets, etc.)
| 14. | /sbin | Core operating system commandsa 
| 15. | /srv | Files held for distribution through web or other servers
| 16. | /sys | A plethora of different kernel interfaces (Linux)
| 17. | /tmp | Temporary files that may disappear between reboots

**🗃 User Directories**

| No. | Directory	| Description |
|:--|:--|:--|
| 1. | /usr | Hierarchy of secondary files and commands
| 2. | /usr/bin | Most commands and executable files
| 3. | /usr/include | Header files for compiling C programs
| 4. | /usr/lib | Libraries; also, support files for standard programs
| 5. | /usr/local | Local software or configuration data; mirrors /usr
| 6. | /usr/sbin | Less essential commands for administration and repair
| 7. | /usr/share | Items that might be common to multiple systems
| 8. | /usr/share/man | On-line manual pages
| 9. | /usr/src | Source code for nonlocal software (not widely used)
| 10. | /usr/tmp | More temporary space (preserved between reboots)

**📂 Variable Directories**

| No. | Directory	| Description |
|:--|:--|:--|
| 1. | /var | System-specific data and a few configuration files
| 2. | /var/adm | Varies: logs, setup records, strange administrative bits
| 3. | /var/log | System log files
| 4. | /var/run | Same function as /run; now often a symlink
| 5. | /var/spool | Spooling (that is, storage) directories for printers, mail, etc.
| 6. | /var/tmp | More temporary space (preserved between reboots)

---
## ⚡️ Processes

### 📖 **Introduction**

<img src="https://drive.google.com/uc?id=1uUcB-YbibkEl-Sazo38PdzXkrO0sApEs" align="middle" width="500px"/>

* An instance of an executing program. 
* When a program is executed, the kernel:
    1. Loads the code of the program into virtual memory
    2. Allocates space for program variables 
    3. Sets up kernel bookkeeping data structures to record info about the process (PID, UID, GID, termination status)
* Process is logically decided into the following segments:
    1. **Text**: instructions of program
    2. **Data**: static variables used by program
    3. **Heap**: area which programs can dynamically allocate extra memory 
    4. **Stack**: memory that grows and shrinks as functions are called and return. It also allocated storage for local variables and function call linkage info. 
  

### 🥞 **Stack versus Heap**

| No. | Parameters | Stack | Heap | 
|:--|:--|:--|:--|
| 1. | Type of data structure |	A stack is a linear data structure.	| Heap is a hierarchical data structure. |
| 2. | Access speed	| High-speed access	| Slower compared to stack |
| 3. | Space management | 	Space managed efficiently by OS so memory will never become fragmented. |	Heap Space not used as efficiently. <br/> Memory can become fragmented as blocks of memory first allocated and then freed.
| 4. | Access | Local variables only |	It allows you to access variables globally.
| 5. | Limit of space size	| Limit on stack size dependent on OS. | Does not have a specific limit on memory size.
| 6. | Resize	| Variables cannot be resized	| Variables can be resized.
| 7. | Memory Allocation	| Memory is allocated in a contiguous block. | Memory is allocated in any random order.
| 8. | Allocation and Deallocation | Automatically done by compiler instructions. |	It is manually done by the programmer.
| 9. | Deallocation |	Does not require to de-allocate variables.	| Explicit de-allocation is needed.
| 10. | Implementation |	A stack can be implemented in 3 ways; arrays, dynamic memory or Linked list.	| Heap can be implemented using array and trees.
| 11. | Main Issue	| Shortage of memory	|Memory fragmentation
| 12. | Flexibility |	Fixed size	| Resizing is possible

<img src="https://drive.google.com/uc?id=1XvXtbvNqk1klSwyfWqbbNLJlp3aLLeZx" align="middle" width="500px"/>

### 🌱 **Process creation and program execution**

1. Parent process executes the `fork()` system call.
1. This duplicates the parent process to create a child process. 
1. Child inherits copies of parent’s data, stack and heap segments (text is shared between both parent and child) 
1. Child process executes the `exec()` system call to execute an entirely new program. 
1. `exec()` destroys existing text, data, stack and heap segments, replacing them with new segments based on source code of program. 

### ❌ **Process termination and termination status**

1. When a child process completed its execution, it will send `SIGCHLD` to its parent. 
1. Parent process sends a `wait()` system call that reads the child’s exit status. 
1. After receiving `wait()` system call, the child’s entry is removed from the process table. 

### **🧟‍♀️ Cause of Zombie Processes**

1. Parent may ignore the SIGCHLD causing finished child processes to still exist in the process table making them zombie processes. 
1. Dead zombie processes have freed up its resources for others (e.g.: CPU, memory) but if a large number of zombie processes exist, the system will run out of PIDs in the process table. 

<img src="https://drive.google.com/uc?id=1lsnq6pZ4zMeo4_O3-rQjRqoTnUy4IpXi" align="middle" width="500px"/>

### 🧟‍♂️ **Zombie process termination**

1. Send a SIGCHLD signal to the parent process of a zombie using: 
    `kill -s SIGCHLD`
2. Parent process sends a `wait()` system call which removes the child zombie process from the process table.
3. If zombie processes are not removed, kill its parent process or restart the service using:
   `kill -9`
4. When parent is killed, all its children will become child process of init process `PID 1`. 
5. Init process sends a `wait()` system call which removes the child zombie processes. 

### 🥇 **Init Process**

1. Mother of all processes with `PID 1`. All processes are created by init using `fork()`. 
2. Its main task is to create and monitor a range of processes required by a running system. 

### 😈 **Daemon Process**

A special-purpose process created and handled by the system with the below special characteristics:
* **Long lived.** It is often started at system boot and remains in existence until system shuts down. 
* **Runs in the background and not interactive in terminal.** It has no controlling terminal that reads input or write outputs. 
* Examples: syslogd - records messages in the system log. httpd - served web pages via HTTP

### 📂 **/proc**

<img src="https://drive.google.com/uc?id=19HJsqY20-d2cnK3Hbnm7-CoM01N13Yz8" align="middle" width="500px"/>

**Summary**
1. A virtual file system that provides an interface to kernel data structures in a form that looks like files and directories on a file system. 
    * /proc is said to be virtual because the files and subdirectories within do not reside on disk but are created "on the fly" in memory as processes access them.
1. Consists of `/proc` files generally in human readable text and parsable by shell scripts. 
1. Exposes a range of kernel information to application programs. Each `/proc/PID` subdirectory contains files and subdirectories that provide information about the running process whose ID matches PID. 
1. Various other files and directories under `/proc` expose system-wide information that programs can read and, in some cases, modify.


**Per-process files via `/proc/PID`**

* Each process on the system is provided a corresponding directory called `/proc/PID` by the kernel. The files and subdirectories within contain information about the process such as:

| File |  Description (process attribute) |
|:--|:--|
|/proc/\$pid/cmdline| Holds the complete command line for the process
|/proc/\$pid/cwd| Symbolic link to the process' current working directory
|/proc/\$pid/environ| Contains the initial environment that was set when the program started
|/proc/\$pid/exe| Symbolic link to the path of the file that was executed
|/proc/\$pid/fd| Subdirectory for listing files that have been opened by the process
|/proc/\$pid/io| Contains I/O statistics for the process
|/proc/\$pid/limits| Displays the process resource limit
|/proc/\$pid/maps| Currently mapped memory regions and access permissions
|/proc/\$pid/mem| Process virtual memory (must `seek()` to valid offset before I/O)
|/proc/\$pid/mounts|Mount points for this process
|/proc/\$pid/root|Symbolic link to root directory
|/proc/\$pid/status|Various information (e.g., process IDs, credentials, memory usage, signals)
|/proc/\$pid/stack| Trace of function calls in the process' kernel stack
|/proc/\$pid/stat| Status information about the process
|/proc/\$pid/task| Directory that contains thread information

**System-wide files via `/proc`**

* Various files and subdirectories under `/proc` provide access to system-wide information.

| File |  Description (process attribute) |
|:--|:--|
|/proc/cmdline| Arguments passed to the Linux kernel during boot time
|/proc/cpuinfo| CPU-related information
|/proc/meminfo| Memory usage statistics
|/proc/filesystems| Listing of filesystems supported by the kernel
|/proc/modules| Kernel modules that are currently loaded
|/proc/partitions| Information about each partition on the system
|/proc/swaps| Information about swap space on the system
|/proc/self| Directory refers to the process accessing the /proc filesystem
|/proc/slabinfo| Information about kernel caches
|/proc/sys| Various files and sub-directories about kernel variables





---
### ❓**What is the difference between a system crash and a kernel panic?**



1. In a `system crash`, the kernel forcefully terminates the process after detecting an abnormal condition such as when the process is trying to access the memory incorrectly. 
1. In a `kernel panic`, the kernel detects an internal fatal error where it is unable to safely recover or continue running the system without high risk of major data loss due to hardware failure or a software bug.

---
## 🧵 Threads

1. A set of processes that share the same virtual memory.
1. Each process can have multiple threads of execution. 
1. Each thread executes the same program code and shares the same data area and heap. 
1. Threads communicate with each other using global variables they share and interprocess communication (signals). 
1. Makes it easy to share data (global variables) between threads and takes advantage of parallel processing on multiprocessor hardware. 

---
### ❓ What is the difference between processes and threads?

| No. |	Process	| Thread
|:--|:--|:--|
|1.|	Process means any program in execution.	|Thread means a segment of a process.
|2.|	The process takes more time to terminate.|	The thread takes less time to terminate.
|3.|	It takes more time for creation.|	It takes less time for creation.
|4.|	It also takes more time for context switching.|	It takes less time for context switching.
|5.	|The process is less efficient in terms of communication.	|Thread is more efficient in terms of communication.
|6.| 	Multiprogramming holds the concepts of multi-process.	|We don’t need multi programs in action for multiple threads because a single process consists of multiple threads.
|7.|	The process is isolated.	|Threads share memory.
|8.|	The process is called the heavyweight process.|	A Thread is lightweight as each thread in a process shares code, data, and resources.
|9.|	Process switching uses an interface in an operating system.	|Thread switching does not require calling an operating system and causes an interrupt to the kernel.
|10.|	If one process is blocked then it will not affect the execution of other processes| 	If a user-level thread is blocked, then all other user-level threads are blocked. 
|11.|	The process has its own Process Control Block, Stack, and Address Space.|	Thread has Parents’ PCB, its own Thread Control Block, and Stack and common Address space.
|12.|	Changes to the parent process do not affect child processes.	|Since all threads of the same process share address space and other resources so any changes to the main thread may affect the behavior of the other threads of the process.

---
## 📞 System Calls

System call is a routine that allows a user application to request the kernel to perform privileged system operations, including device I/O.

|No. | Syscall	| Function |
|:--|:--|:--|
| 1. | read	| read bytes from a file descriptor (file, socket)
| 2. | write	| write bytes from a file descriptor (file, socket)
| 3. | open | open a file (returns a file descriptor)
| 4. | close	| close a file descriptor
| 5. | fork	| create a new process (current process is forked)
| 6. | exec	| execute a new program
| 7. | connect	| connect to a network host
| 8. | accept	| accept a network connection
| 9. | stat	| read file statistics
| 10. | ioctl	| set I/O properties, or other miscellaneous functions
| 11. | mmap	| map a file to the process memory address space
| 12. | brk	| extend the heap pointer


1. A controlled entry point into the kernel allowing a process to request the kernel to perform some action on the process’s behalf. 
2. System calls changes the processor system from user mode to kernel mode so CPU can access protected kernel memory. 

<img src="https://drive.google.com/uc?id=1T52206noDKOrL3VLf1FP8cM71IYwMJRm" align="middle" width="500px"/>

---
### 💡 Overview of `fork()`, `exit()`, `wait()` and `execve()`

<img src="https://drive.google.com/uc?id=1AsTYDcOG6NhWRrsSjKw7CEauXFV6f0tw" align="middle" width="500px"/>

| Factors for Differentiation	|fork()| exec()
|:--|:--|:--|
|Invoking|	`fork()` creates a new duplicate child process of the process that invoked `fork()`	|`exec()` replaces a process that invokes it with a new process provided in its parameter.
|Process id|	The child process and parent process have unique process id. 	|The new process and the replaced process have the same process id.
|Execution|	The parent and child process start simultaneous execution from the instruction just after `fork()`.	|The currently running process is terminated and the `exec()` start execution of the new process from its entry point.
|Arguments|	No arguments are passed to `fork()` system call.	|Basically, three or more arguments are passed to the `exec()` system call.
|Format|	`Pid=fork();`	|`exec(cont char *filename, char* const argv[], char* const envp[])`

### 🍴 `fork()`

1. The `fork()` system call allows one process, the parent, to create a new process, the child. 
1. This is done by making the new child process an (almost) exact duplicate of the parent: the child obtains copies of the parent’s stack, data, heap, and text segments.
1. The term fork derives from the fact that we can envisage the parent process as dividing to yield two copies of itself.

### ⚡ `exec()`

1. The `execve(pathname, argv, envp)` system call loads a new program (pathname, with argument list argv, and environment list envp) into a process’s memory. 
1. The existing program text is discarded, and the stack, data, and heap segments are freshly created for the new program. 
1. This operation is often referred to as execing a new program. Later, we’ll see that several library functions are layered on top of `execve()`, each of which provides a useful variation in the programming interface. 
1. Where we don’t care about these interface variations, we follow the common convention of referring to these calls generically as `exec()`, but be aware that there is no system call or library function with this name.

### ✋ `wait()` 
* The wait system call has two purposes. First, if a child of this process has not yet terminated by calling `exit()`, then `wait()` suspends execution of the process until one of its children has terminated. 
* Second, the termination status of the child is returned in the status argument of `wait()`.

The `wait()` system call waits for one of the children of the calling process to terminate and returns the termination status of that child in the buffer pointed to by status.
The `wait()` system call does the following:

1. If no (previously unwaited-for) child of the calling process has yet terminated, the call blocks until one of the children terminates. If a child has already terminated by the time of the call, `wait()` returns immediately.

2. If status is not NULL, information about how the child terminated is returned in the integer to which status points.

3. The kernel adds the process CPU times and resource usage statistics to running totals for all children of this parent process.

4. As its function result, `wait()` returns the process ID of the child that has terminated.

### 🏃 `exit()`
1. The exit(status) library function terminates a process, making all resources (memory, open file descriptors, and so on) used by the process available for subsequent reallocation by the kernel. 
1. The status argument is an integer that determines the termination status for the process. Using the `wait()` system call, the parent can retrieve this status.

### ❓**What exactly happens when you type “ls” into the shell?**

**Brief Explanation** 
1. When you run `ls`, the shell searches in its path for an executable named `ls` in /bin/ls
1. When it finds it, the shell will forks off a copy of itself using the `fork()` system call. **Note:** System calls involve mode switching from user space to kernel space allowing the `ls` command to request the kernel to perform privileged system operations.
1. If the fork succeeds, then in the child process the shell will run `exec /bin/ls` which will replace the copy of the child shell with itself. 
1. Any parameters that are passed to `ls` are done so by `exec()`

**Detailed Explanation**

*Step 1: `Find executable binary`*
1. The shell reads what you have typed using the `getline()` function and tokenizes the line using the  `strtok()` function.
1. It also checks if the first token of `ls` is a Shell `alias` or not. 
1. If the command is not a built-in (internal) function, the shell will find the `PATH` variable in the directory where it holds the absolute paths for all the executable binary files.

*Step 2: `fork()`*
1. Once it finds the binary for `ls` located at `/bin/ls`, the program is loaded in memory and a system call `fork()` is made. 
1. This creates the child process `ls` and the shell will be the parent process. 
1. `fork()` returns `0` to the child process so it knows it has to act as a child and returns the PID of the child to the parent process (e.g. the shell).

*Step 3: `execve()`*
1. Next, the `ls` process executes the system call `execve()` that provides a new address space with the program to run.
1. Now, `ls` can start running its program. The `ls` utility uses a function to read the directories and files from the disk by consulting the underlying filesystem’s inode entries.

*Step 4: `exit()`*
1. Once the `ls` process has finished executing, it will perform the `exit()` system call with an integer `0` that denotes a normal execution and the kernel will free up its resources.
1. The child process also sends a `SIGCHLD` signal to the parent process to indicate that the child process has ended.
1. This saves the exit status of the child process so that the parent process can identify which child process (by process ID) ended and its exit status.


*Step 5: `wait()`*
1. The parent process receives the `SIGCHLD` signal and reads the exit status of the child process. The parent then performs `wait()` system call which reaps off the child process entry from the process table. This prevents accumulation of any zombie processes in the process table.

* Note: you can use `strace ls` to dig deeper into the system calls



---
## 🍿 Kernel Mode versus User Mode

<img src="https://drive.google.com/uc?id=1CeJn5xFA7m16dDkrMieczhYUhIHu0hYF" align="middle" width="500px"/>

1. In user mode, CPU can access only memory that is marked as user space; attempts to access memory in kernel space result in hardware exception. 
1. In kernel mode, CPU can access both user and kernel memory space. 
1. This ensures that user processes are not able to access the instructions and data structures of the kernel, or perform operations that would adversely affect the operation of the system. 

*User and Kernel Space Flow*
1. Every process in linux utilizes its own separate virtual space.
1. In a 32-bit architecture Linux system, user space address space corresponds to lower 3GB of virtual space and kernel space the upper 1GB.
1. The kernel space virtual address space is shared between all the processes.
1. When a process is active, it can either be running in "user mode" or "kernel mode".
1. If a process is running in user mode, it means that the CPU is running the user space side of code.
1. A process running in the user mode has limited capability and is controlled by a flag in the CPU.
1. Eventhough the kernel memory is present in the process's memory map, the user space code is not allowed to access the kernel space code.
1. When a process wants to do something other than move data around in its own (user space) virtual memory (e.g.: opening a file), it must make a system call to communicate with the kernel space.
1. Each CPU architecture has it's unique way of making a system call but the basic remains the same (e.g.:
A magic instruction is executed, the CPU turns on the "privileged mode" flag, and jumps to a special address in kernel space, the "system call entry point".)
1. Now when the system call has reached the kernel space, the process is running in kernel mode and executes instructions from the kernel space memory.
1. Taking the same example of open system call, to find the requested file, the kernel may consult with filesystem drivers (to figure out where the file is) and block device drivers (to load the necessary blocks from disk) or network device drivers and protocols (to load the file from a remote source).
1. These drivers can be either built in or can be loaded as module but the key point that remains that they are the part of kernel space.
1. Loading a module is done with a system call that asks the kernel to copy the module's code and data into kernel space and run its initialization code in kernel mode.
1. If the kernel can't process the request then the process is made to sleep by the kernel and when the request is complete, the system call returns back to the user space.
1. Returning back to user mode means restoring the CPU registers to what they were before coming to Kernel Mode and changing the CPU privilege level to non-privilege .
1. Apart from system calls, there are other things that take CPU to kernel mode such as: 
    * **Page faults:** If the process tries to access a virtual memory address that doesn't have a physical address assigned to it then the CPU enters the Kernel mode  and jumps to page fault handler and the kernel sees whether the virtual addresss is valid or not and depending upon this it either tries to create a physical page for the given virtual address or if it can't then sends a segmentation fault signal (SIGSEGV).
    * **Interrupts:** When the CPU receives some interrupt from the hardware then it jumps to the kernel mode and executes the interrupt handler and when the kernel is finished handing the interrupt the the code return to the user space where it was executing.

---
## 🚦 Signals

### 📖 **Introduction**

* An interprocess communication. Signals are sent to a process by the kernel, by another process. 
* Kernel may send a signal to a process when:
    1. User typed the interrupt character (CTRL+C) on the keyboard 
    2. One of the process’s children had terminated 
    3. A timer set to the process has expired
    4. Process attempted to access an invalid memory address 

| Signal | Value | Action | Comment |
|:--|:--|:--|:--|
| SIGHUP   |     1  |     Term  |  Hangup detected on controlling terminal or death of controlling process |
| SIGINT    |    2    |   Term  |  Interrupt from keyboard |
| SIGQUIT   |    3     |  Core  |  Quit from keyboard |
| SIGILL     |   4     |  Core  |  Illegal Instruction |
| SIGABRT     |  6   |    Core  |  Abort signal from abort(3) |
| SIGFPE     |   8    |   Core   | Floating-point exception 
| SIGKILL  |     9     |  Term   | Kill signal
| SIGSEGV   |   11   |    Core  |  Invalid memory reference
| SIGPIPE   |   13   |    Term  |  Broken pipe: write to pipe with no readers; see pipe(7)
| SIGALRM  |    14   |    Term  |  Timer signal from alarm(2)
| SIGTERM  |    15    |   Term  |  Termination signal
| SIGUSR1  | 30,10,16  |  Term  |  User-defined signal 1
| SIGUSR2  | 31,12,17  |  Term  |  User-defined signal 2
| SIGCHLD  | 20,17,18 |   Ign   |  Child stopped or terminated
| SIGCONT  | 19,18,25  |  Cont   | Continue if stopped
| SIGSTOP   | 17,19,23  |  Stop  |  Stop process
| SIGTSTP  | 18,20,24  |  Stop  |  Stop typed at terminal
| SIGTTIN  | 21,21,26  |  Stop  |  Terminal input for background process
| SIGTTOU |  22,22,27 |    Stop |  Terminal output for background process

**Quick Reference**

|Name |Command |
|:--|:--|
|kill -l|	List all signals|
|Critical signals|	The signals SIGKILL and SIGSTOP cannot be caught, blocked, or ignored.|
|SIGKILL, SIGTERM|	Send kill signal to a given process. kill -term \$pid, kill -9 \$pid|
|SIGHUP|	Reload or restart a process. kill -HUP $pid|
|SIGINT|	Interrupt a process|
|SIGCHILD|	When a child process terminates, instruct OS to cleanup the resource|
|SIGSTOP|	Suspend a process|
|SIGALRM|	Raised when a time interval specified in a call to the alarm or alarmd function expires|
|SIGUSR1, SIGUSR2|	User defined signals|
|SIGSTOP vs SIGTSTP|	SIGSTOP can’t be ignored by the targetted process, but SIGTSTP can|
|What happens, when pressing Ctrl-C|	Kernal translates the ASCII character(^C) to SIGINT signal|


### 🚨 **SIGTERM 15**

1. SIGTERM signal is safest way to kill the process. 
1. It instructs the process to terminate and gives the programs or services time to wrap things up before be shutdown such as writing to log files or closing opened connections that were created by the process.

### 🚨 **SIGKILL 9**

1. SIGKILL signal is an unsafe way for killing a process because it ends process without saving anything.
1. This signal is sent by a process in order to terminate the receiving process immediately. 
1. This means that the termination is abnormal and not graceful. 
1. This signal is used in cases where a process hangs or is to be terminated immediately. 
1. For example, this signal is used by OS kernel sometimes when the system is shutdown.
1. If the process is hung badly, this option will force it to stop, but the child processes orphan and later convert to zombies.

### 🚨 **SIGSTOP 23**

1. Sent by a process in order to halt a program so that it can be debugged. 
1. Suppose if the programs were given the capability of handling this signal and a program accidentally chooses to ignore this signal, then that program cannot be debugged ever.

### 🚨 **SIGCHLD 18**

When a child process created by a parent process ends, the child:
1. Sends a SIGCHLD signal to the parent process to indicate that the child process has ended
1. Saves the exit status of the child process so that the parent process can identify which child process (by process ID) ended and its exit status.

### 🚨 **SIGHUP 1**

1. This signal restarts the process. 
1. After a restart, the process will have exactly the same PID that it had before. 
1. This is a very useful option for restarting a service, such as a website, for which an end user made changes to a configuration file.

### 🚨 **SIGINT 2**

1. This signal interruipts the process. 
1. This signal sends a CTRL-C key sequence to the process.

---
### 🛑 **Interrupts** 

1. An interrupt is an event that alters the normal execution flow of a program and can be generated by hardware devices or even by the CPU itself. 
1. When an interrupt occurs the current flow of execution is suspended and interrupt handler runs. 
1. After the interrupt handler runs the previous execution flow is resumed.

1. Interrupts can be grouped into two categories based on the source of the interrupt. 
    * **Synchronous:** Generated by executing an instruction. Synchronous interrupts, usually named exceptions, handle conditions detected by the processor itself in the course of executing an instruction. Divide by zero or a system call are examples of exceptions.

    * **Asynchronous:** Generated by an external event. Asynchronous interrupts, usually named interrupts, are external events generated by I/O devices. For example a network card generates an interrupts to signal that a packet has arrived.
1. Most interrupts are maskable, which means we can temporarily postpone running the interrupt handler when we disable the interrupt until the time the interrupt is re-enabled. However, there are a few critical interrupts that can not be disabled/postponed.


---
## 🗂 Linux File Systems

---
### 💾 Mount Points

<img src="https://drive.google.com/uc?id=1M_n4Ke1mnsdCGavEHfO6-mfbfQoigw88" align="middle" width="500px"/>

1. A mount point is simply a directory, like any other, that is created as part of the root filesystem. 
    * For example, the home filesystem is mounted on the directory /home. 
1. The Linux root filesystem is mounted on the root directory (/) very early in the boot sequence. 
1. Other filesystems are mounted later, by the Linux startup programs, either rc under SystemV or by systemd in newer Linux releases. 
1. Mounting of filesystems during the startup process is managed by the `/etc/fstab` configuration file. 
1. An easy way to remember that is that fstab stands for "file system table," and it is a list of filesystems that are to be mounted, their designated mount points, and any options that might be needed for specific filesystems.

---
### ⚡ mount command
1. Filesystems are mounted on an existing directory/mount point using the `mount` command. 
1. In general, any directory that is used as a mount point should be empty and not have any other files contained in it. 

`$ mount device directory`

>* This command attaches the file system on the named device into the `directory` hierarchy at the specified directory—the file system’s mount point.
>* It is possible to change the location at which a file system is mounted—the file system is unmounted using the umount command, and then mounted once more at a different point.

---
### 🗃 Logical Volume Manager

* A device mapper framework that provides logical volume management for the Linux kernel. 
* It is mainly used to create single logical volumes of multiple physical volumes or entire hard disks allowing for dynamic volume resizing.


<img src="https://drive.google.com/uc?id=1F10EynD_6K9H0NiDIXP1m0WDj1ENViX-" align="middle" width="500px"/>

<img src="https://drive.google.com/uc?id=19uBk_mpFYLJoP-6xq-os4-56pHK68MBK" align="middle" width="500px"/>

1. Volume groups (VGs) can be resized online by absorbing new physical volumes (PVs) or ejecting existing ones.
1. Logical volumes (LVs) can be resized online by concatenating extents onto them or truncating extents from them.
1. LVs can be moved between PVs.

---
### ☁ Virtual File System

<img src="https://drive.google.com/uc?id=1c0ig-zWbVTWiQHItfDyGGZTTyksk26Wv" align="middle" width="500px"/>

1. Defines a generic interface for file-system operations and provides a single set of commands for the kernel, and developers, to access all types of filesystems. 
    * The virtual filesystem software calls the specific device driver required to interface to the various types of filesystems. 
    * All programs that work with files specify their operations in terms of this generic interface. 
    * Each file system provides an implementation for the VFS interface.
1. Under this scheme, programs need to understand only the VFS interface and can ignore details of individual file-system implementations.
1. The VFS interface includes operations corresponding to all of the usual system calls for working with file systems and directories, such as `open()`, `read()`, `write()`, `lseek()`, `close()`, `truncate()`, `stat()`, `mount()`, `umount()`, `mmap()`, `mkdir()`, `link()`, `unlink()`, `symlink()`, and `rename()`.
1. The VFS abstraction layer is closely modeled on the traditional UNIX file-system model. 

### 💿 Filesystem-specific Device Drivers
1. Interprets the standard set of filesystem commands to ones specific to the type of filesystem on the partition or logical volume.




---
### 🗄 Filesystem Types
1. Linux supports reading around 100 partition types; it can create and write to only a few of these. 
1. In this context we are talking about filesystems in terms of the structures and metadata required to store and manage the user data on a partition of a hard drive or a logical volume. 
1. The complete list of filesystem partition types recognized by the Linux can be listed using the `fdisk` command.
1. The main purpose in supporting the ability to read so many partition types is to allow for compatibility and at least some interoperability with other computer system's filesystems. 
1. Filesystem examples with Fedora distro:
    * cramfs
    * ext2
    * fat
    * vfat
    * ntfs
    * gfs2
    * hfsplus
    * minix
    * msdos

  Journaling File Systems:
    * reiserfs
    * ext3
    * ext4
    * btrfs
    * xfs

---
### 📖 Journaling File Systems

**Disadvantages of a traditional UNIX file system**
* The ext2 file system is a good example of a traditional UNIX file system, and suffers from a classic limitation of such file systems:
    1. After a system crash, a file-system consistency check ( fsck) must be performed on reboot in order to ensure the integrity of the file system. 
    1. This is necessary because, at the time of the system crash, a file update may have been only partially completed, and the file-system metadata (directory entries, i-node information, and file data block pointers) may be in an inconsistent state, so that the file system might be further damaged if these inconsistencies are not repaired. 
    1. A file-system consistency check ensures the consistency of the file-system metadata. Where possible, repairs are performed; otherwise, information that is not retrievable (possibly including file data) is discarded.
* The problem is that a consistency check requires examining the entire file system. 
    * On a small file system, this may take anything from several seconds to a few minutes. 
    * On a large file system, this may require several hours, which is a serious problem for systems that must maintain high availability (e.g., network servers).

**Advantages of a journaling file system**
* Journaling file systems eliminate the need for lengthy file-system consistency checks after a system crash. 
    1. A journaling file system logs (journals) all metadata updates to a special on-disk journal file before they are actually carried out. 
    1. The updates are logged in groups of related metadata updates (transactions). 
    1. In the event of a system crash in the middle of a transaction, on system reboot, the log can be used to rapidly redo any incomplete updates and bring the file system back to a consistent state. (To borrow database parlance, we can say that a journaling file system ensures that file metadata transactions are always committed as a complete unit.) 
    1. Even very large journaling file systems can typically be available within seconds after a system crash, making them very attractive for systems with highavailability requirements.
* The most notable disadvantage of journaling is that it adds time to file updates, though good design can make this overhead low.

---
### 📔 Filesystem Structure

<img src="https://drive.google.com/uc?id=1zJF53hz1sCYQMwjMxx5eNq7h6Wp-MtxD" align="middle" width="500px"/>

A file system contains the following parts:
1. **Boot block** 
    * This is always the first block in a file system. The boot block is not used by the file system; rather, it contains information used to boot the operating system. 
    * Although only one boot block is needed by the operating system, all file systems have a boot block (most of which are unused).
1. **Superblock**
    * This is a single block, immediately following the boot block, which contains parameter information about the file system, including:
        1. the size of the i-node table;
        1. the size of logical blocks in this file system; and
        1. the size of the file system in logical blocks.
    * Different file systems residing on the same physical device can be of different types and sizes, and have different parameter settings (e.g., block size). 
    * This is one of the reasons for splitting a disk into multiple partitions.
1. **I-node table** 
    * Each file or directory in the file system has a unique entry in the i-node table. 
    * This entry records various information about the file. 
    * The i-node table is sometimes also called the i-list.
1. **Data blocks**
    * The great majority of space in a file system is used for the blocks of data that form the files and directories residing in the file system.

---
## 🗃 File Structures

### 🔏 **File Permissions**

<img src="https://drive.google.com/uc?id=1iC_w-744cH91tMcyWTBtjTLvsKgPgpQ0" align="middle" width="250px"/>






1. `chown`
1. `chmod`
1. `chattr`
1. `umask`
1. `getfacl`
1. `setfacl`

---
### 🥎 **Soft Link Files**
1. In Linux, a soft link, also known as a symbolic link, is a special sort of file that points at a different file. 
    * In Windows vocabulary, you could think of it like a shortcut. 
1. Because the connection is a logical one, and not a duplication, soft links can point at entire directories or link to files on remote computers. Hard links cannot do this.
1. Soft link files do not share the same inode hence they can be created and referenced between two different filesystems.
1. Symbolic links are created using `ln -s` and removed using `rm`.
```yaml
$ sudo ln -s archived/secure /var/data/secure
# links /var/data/secure to /var/data/archived/secure with a relative path. 
# It creates the symbolic link /var/data/secure with a target of archived/secure, as demonstrated by this output from ls:
$ ls -l /var/data/secure 
lrwxrwxrwx 1 root root 18 Aug 3 12:54 /var/data/secure -> archived/secure
```

### 🧱 **Hard Link Files**
1. In Linux, a hard link is equivalent to a file stored in the hard drive – and it actually references or points to a spot on a hard drive. 
1. A hard link is a mirror copy of the original file. The distinguishing characteristic of a hard link from a soft link is that deleting the original file doesn't affect a hard link, while it renders a soft link inoperable.
1. Hard link files share the same inode hence they cannot be created in two different filesystems. 
1. Hard links are created using `ln` and removed using `rm`.

### **❓ What is the difference between soft link and hard link ?**
1. Hard link shares the same inode number as the source while the soft link uses a different inode number.
1. As a result, if the source file is moved, the soft link will be broken but hard link will still working fine.
1. The hard link is only valid on the same filesystem while soft link is not.

### 📛 **File Descriptors**

<img src="https://drive.google.com/uc?id=1f6YnnwLBDXqCShQ_SRjyFYtXSOBpCjBE" align="middle" width="500px"/>

When a process is created, the file consist of a file pointer array where the first three bits correspond to `0: standard input stream (stdin)`, `1: standard output stream (stdout)`, and `2: standard error stream (stderr)`.

1. A file descriptor is a reference (also called a handle) used by the kernel to access a file. 
1. A file descriptor table is created for all process and tracks all files opened by the process. 
1. A unique index number (file descriptor number) is assigned to a file when it is opened by a process.
1. When a process is created, the file descriptor numbers 0, 1, and 2 are reserved for the stdin, stdout, and stderr devices, respectively:
    * `fd0`: Standard input device (stdin).
    * `fd1`: Standard output device (stdout).
    * `fd2`: Standard error device (stderr).
    * `fd255`: The bash file descriptor 255 keeps track of controlling terminal information.

---
### 📚 **Inodes**

<img src="https://drive.google.com/uc?id=12w9YXlsEnUu7yq-SWJf4X6U7xBGFMNsL" align="middle" width="600px"/>

1. A data structure storing the properties of a file or directory. 
2. The following information is stored in inode:
    * **File type**: regular file, directory, pipe etc.
    * **Permissions** to that file: read, write, execute
    * **Link count**: The number of hard link relative to an inode
    * **User ID**: owner of file
    * **Group ID**: group owner
    * **Size of file**: or major/minor number in case of some special files
    * **Time stamp**: access time, modification time and (inode) change time
    * **Attributes**: immutable' for example
    * **Access control list**: permissions for special users/groups
    * Link to location of file
    * Other metadata about the file
3. `ls command` is used to list file and directories information. The `-i` option with ls displays the inode number of each file. First column gives the inode number. 

---
### **❓What is an inode?** 
1. An inode is a data structure in Unix that contains metadata about a file. 
1. These following inode attributes which may be retrieved by the stat system call:
    * mode
    * owner (UID, GID)
    * size
    * atime, ctime, mtime
    * acls
    * blocks list of where the data is

* Note: The filename is present in the parent directory’s structure, not in the inode.



### **❓How do you delete a file with special characters?**
1. `ls -i` to list the inode number of file with special characters (e.g.: `"ab*`)
```bash
>> 1448239 "ab*
```
1. Then, run `find . -inum 1448239 -exec rm -i {} \`
    * We used the `find` command to specify the Inode number of the file to delete

---
## 🧠 Memory

### 💾 **Virtual memory versus swap space**

<img src="https://drive.google.com/uc?id=15kd6Anqa-CH2qfWfs2pwlI4xLRlWrrVI" align="middle" width="700px"/>

<img src="https://drive.google.com/uc?id=1vL_K7O_vkI2m_Ue3s1SAMmt27vxf8jcP" align="middle" width="750px"/>

---
### ❓[**What is a load average?**](https://www.brendangregg.com/blog/2017-08-08/linux-load-averages.html)

1. The load average is an average number of threads over a time period (last 1, 5, 15 mins) that “compete for CPU” on classic unixes or “either compete for CPU or wait in an uninterruptible sleep state” on Linux
1. They indicate the number of tasks (threads) from processes wanting to run.
1. This measures demand, which can be greater than what the system is currently processing. 
1. Most tools show three averages, for 1, 5, and 15 minutes:

```bash
$ uptime
 16:48:24 up  4:11,  1 user,  load average: 25.25, 23.40, 23.46
top - 16:48:42 up  4:12,  1 user,  load average: 25.25, 23.14, 23.37
$ cat /proc/loadavg 
25.72 23.19 23.35 42/3411 43603
```

**Some interpretations:**
* If the averages are 0.0, then your system is idle.
* If the 1 minute average is higher than the 5 or 15 minute averages, then load is increasing.
* If the 1 minute average is lower than the 5 or 15 minute averages, then load is decreasing.
* If they are higher than your CPU count, then you might have a performance problem (it depends).

---
## 🪵 Logs

### 🗃 Type of log files

<img src="https://drive.google.com/uc?id=1kkDGvuyL6xE87_LAjoOWGQv3gM-WTbrd" align="middle" width="500px"/>

### ♻ Regular Expressions

<img src="https://drive.google.com/uc?id=1ogXJqcCrcX2cjzQ0gepi304eU2gaTjJ8" align="middle" width="500px"/>

---
# [🛠 Linux Troubleshooting](https://linkedin.github.io/school-of-sre/level102/system_troubleshooting_and_performance/troubleshooting/)

<img src="https://drive.google.com/uc?id=1FHMyBUV4AP0VE3Z7p-tuSSaNxa4xHS2J" align="middle" width="500px"/>

## 💿 Disk Management

### ❓**What to do when a Linux system is running out of available inodes?**

1. If there are free available storage but the error *“no space left on device”* keeps showing, this is due to inode saturation where there is insufficient space left in the inode table. 
1. Determine where all your inodes are being spent. Often they’re being consumed by millions of small files created by a misconfigured system
1. Run the following command to identify the directory and system that’s using all your inodes: 
`du --inodes -d 3 /  | sort -n | tail`
1. Or start cleaning out unexpected files (e.g.: ton of cache files)
1. Or reformat the disk and assign more of the disk to maintain the inodes table

### ❓ **How to troubleshoot "filesystem is full" error message when there is free space in `df`?**

1. Use `df -i` to see if there are zero IFree. 
1. Otherwise, check if the deleted files are still in use using `lsof` and restart those processes.


---
## ⚡ Processes

### **❓How do you troubleshoot a Linux instance if none of the external commands work?**

1. Use `help` to display a list of all the built-in commands in shell.
1. Referring to the `/proc` table below, use `read $(</proc/${PID}/${procInformation})` to obtain information on a specific process.  

**Per-process files via `/proc/PID`**

* Each process on the system is provided a corresponding directory called `/proc/PID` by the kernel. The files and subdirectories within contain information about the process such as:

| File |  Description (process attribute) |
|:--|:--|
|/proc/\$pid/cmdline| Holds the complete command line for the process
|/proc/\$pid/cwd| Symbolic link to the process' current working directory
|/proc/\$pid/environ| Contains the initial environment that was set when the program started
|/proc/\$pid/exe| Symbolic link to the path of the file that was executed
|/proc/\$pid/fd| Subdirectory for listing files that have been opened by the process
|/proc/\$pid/io| Contains I/O statistics for the process
|/proc/\$pid/limits| Displays the process resource limit
|/proc/\$pid/maps| Currently mapped memory regions and access permissions
|/proc/\$pid/mem| Process virtual memory (must `seek()` to valid offset before I/O)
|/proc/\$pid/mounts|Mount points for this process
|/proc/\$pid/root|Symbolic link to root directory
|/proc/\$pid/status|Various information (e.g., process IDs, credentials, memory usage, signals)
|/proc/\$pid/stack| Trace of function calls in the process' kernel stack
|/proc/\$pid/stat| Status information about the process
|/proc/\$pid/task| Directory that contains thread information

**System-wide files via `/proc`**

* Various files and subdirectories under `/proc` provide access to system-wide information.

| File |  Description (process attribute) |
|:--|:--|
|/proc/cmdline| Arguments passed to the Linux kernel during boot time
|/proc/cpuinfo| CPU-related information
|/proc/meminfo| Memory usage statistics
|/proc/filesystems| Listing of filesystems supported by the kernel
|/proc/modules| Kernel modules that are currently loaded
|/proc/partitions| Information about each partition on the system
|/proc/swaps| Information about swap space on the system
|/proc/self| Directory refers to the process accessing the /proc filesystem
|/proc/slabinfo| Information about kernel caches
|/proc/sys| Various files and sub-directories about kernel variables

3. If there are too many files opened and all resources are exhausted, use the below command to display all processes with all their associated file descriptors (fd):
```bash
$ for fd in /proc/[0-9]*/fd/*; do echo $fd; done
/proc/1/fd/0
/proc/1/fd/1
/proc/1/fd/2
/proc/1/fd/255
/proc/1/fd/3
```

4. To see the last comand ran on the system without using `cat`, use `read $(</proc/${PID}/cmdline)`

### ❓**How to find configuration files related to a process?**

* Use `lsof -p ${PID}` to list files opened by the process.

### ❓ [**How do you troubleshoot “fork failed: Resource temporarily unavailable”?**](https://access.redhat.com/solutions/22105)

1. This means the operating system has limited your ability to `fork` stopping you from creating new threads or processes.
1. This issue can occur when the `ulimit` settings have been set too low or the maximum number of processes has been reached preventing new child processes from spawning.
    * The max value of processes can be found in `/proc/sys/kernel/pid_max`
1. To resolve this, you will need to log in with an unlimited account, typically `root`, to modify the default ulimit settings that may be limiting your user.
```python
ulimit -n 100 -u 35 # limits the number of open files descriptors to 100 and the number of processes to 35
```
1. You can also edit the `ulimit` setting using the below files
    * /etc/bashrc
    * /etc/profile
1. You can also use `/etc/sysctl.conf` (see: `man sysctl.conf`) to increase the kernel limits, for example:
```bash
kern.maxprocperuid=1000
kern.maxproc=2000
```
    * Alternatively, you can run `sysctl -w kernel.pid_max=131072`

---
## 🌎 Networking

### ❓**How to count the total number of TCP connections in the system?**

1. Use `ss -s` (summary statistics) 

```bash
$ ss -s
Total: 1786 (kernel 0)
TCP:   52 (estab 12, closed 14, orphaned 0, synrecv 0, timewait 3/0), ports 0
Transport Total     IP        IPv6
*         0         -         -        
RAW       2         1         1        
UDP       36        25        11       
TCP       38        28        10       
INET      76        54        22       
FRAG      0         0         0  
```

2. Read the `sockstat` file in the `/proc` filesystem

```bash
$ cat /proc/net/sockstat
sockets: used 1783
TCP: inuse 27 orphan 0 tw 1 alloc 48 mem 16
UDP: inuse 23 mem 85
UDPLITE: inuse 0
RAW: inuse 1
FRAG: inuse 0 memory 0
```

3. Read the `tcp` file in the `/proc` filesystem

```bash
$ awk 'BEGIN{printf("%6s %6s\n","STATE", "COUNT")} NR>1 {count[$4]++} END{for(key in count){printf("%6s %6s\n",key,count[key])}}' /proc/net/tcp
 STATE  COUNT
    08      1
    01     12
    0A     15
    06      3
```





### ❓ **How do you check for open ports in the linux server?**


* Use `netstat -tulpn`
```bash
$ netstat -tulpn
Active Internet connections (only servers)
Proto Recv-Q Send-Q Local Address           Foreign Address         State       PID/Program name
tcp        0      0 0.0.0.0:111             0.0.0.0:*               LISTEN      669/rpcbind     
tcp        0      0 0.0.0.0:22              0.0.0.0:*               LISTEN      770/sshd        
tcp6       0      0 :::111                  :::*                    LISTEN      669/rpcbind     
tcp6       0      0 :::22                   :::*                    LISTEN      770/sshd        
udp        0      0 0.0.0.0:68              0.0.0.0:*                           576/dhclient    
udp        0      0 0.0.0.0:111             0.0.0.0:*                           669/rpcbind     
udp        0      0 10.0.0.34:123           0.0.0.0:*                           800/ntpd        
udp        0      0 127.0.0.1:123           0.0.0.0:*                           800/ntpd        
udp        0      0 0.0.0.0:123             0.0.0.0:*                           800/ntpd        
udp        0      0 0.0.0.0:845             0.0.0.0:*                           669/rpcbind     
udp6       0      0 :::111                  :::*                                669/rpcbind     
udp6       0      0 fe80::42:aff:fe00:2:123 :::*                                800/ntpd        
udp6       0      0 ::1:123                 :::*                                800/ntpd        
udp6       0      0 :::123                  :::*                                800/ntpd        
udp6       0      0 :::845                  :::*                                669/rpcbind   
```

---
## 🔥 Performance Analysis 

### **❓How do you troubleshoot a slow server?**

**Step 1: ⚡ Process CPU Analysis**
1. Run `ps auxww` or `top` to look for obvious runaway processes (e.g.: processes that take more than 50% of the CPU)
1. Otherwise, check if two or three processes are getting at least 10% CPU and inspect the **load average** with `uptime`.
1. Finally, use `vmstat` or `top` to check whether the CPU is ever idle.

**Step 2: 🧠 Virtual Memory Analysis**
- Run `vmstat 1` to see how much paging is going on. 

**Step 3: 💽 Disk Analysis**
- Inspect the disk activity using `iostat -xz 1`

**Step 4: 🔎 Narrow Down Suspects Processes**
1. Use `ps` to narrow down suspects and use `kill -STOP` to suspend the process.
1. Use `ulimit -m 32000000` to cause all subsequent commands that the user runs to have their use of physical memory limited to 32MB.

**Step 5: 🌎 Network Analysis**
- Use `sar -n DEV 1` to check network interface throughput.

---
### 🧰 [Performance Analysis Cheat Sheet](https://netflixtechblog.com/linux-performance-analysis-in-60-000-milliseconds-accc10403c55)

```bash
uptime
dmesg | tail
vmstat 1
mpstat -P ALL 1
pidstat 1
iostat -xz 1
free -m
sar -n DEV 1
sar -n TCP,ETCP 1
top
```

---
## 🧯 Linux Troubleshooting Interview Questions

### **❓ A user is unable to `ssh` to a remote server, what is the cause?**

**Insufficient permissions in ssh private key**
1. Run `chmod 400 myPrivateKey.pem`
2. Then, `ssh -i myPrivateKey.pem user@192.168.0.1`

**Wrong ssh configurations in `sshd` config file**
1. Read the ssh config file using `vim /etc/ssh/sshd_config`
2. Check the ssh configurations such as:
    * `Port` - Specifies the port number that sshd listens on. The default is 22.
    * `AddressFamily` - Specifies which address family should be used by sshd. Valid arguments are any (the default), inet (use IPv4 only), or inet6 (use IPv6 only).
    * `PermitRootLogin` - Specifies whether root can log in using ssh(1).
    * `DenyUsers` - A list of users who are denied access
    * `PublicKeyAuthentication` - to allow or deny ssh authentication using public keys
    * `AuthorizedKeysFile` - Specifies the file that contains the public keys used for user authentication. (e.g.: `.ssh/authorized_keys`)
    * `PasswordAuthentication` - Specifies whether password authentication is allowed. The default is yes.
    
Note: Make sure the permissions to the `sshd` directory is enough (`drwx--x--x`)

---
### **❓ Root user created a file called `/tmp/abc/` with 700 permissions. You want another user to have full access to this file, how do you do this?**

1. Create a file using `touch /tmp/abc`
2. Check the permissions of this file using `ll /tmp/abc`

```bash
-rw-r--r-- /tmp/abc # 644 permissions is set by default
```

3. Create a user using `useradd ken` and set a password using `passwd ken`
4. Change the permissions of the file `chmod 700 /tmp/abc` and check again `ll /tmp/abc`
5. As ken, try to access the file. You will be prompted 'Permission denied'.
```bash
> su - ken
> cat /tmp/abc
cat: /tmp/abc: Permission denied
> logout
```

6. Elevate the permissions of the user for that file using `setfacl -m u:ken:rwx /tmp/abc`. Use `-R` flag to recursively apply permissions to all files within directory.
7. Check your permissions again as that user using `getfacl /tmp/abc`
```bash
> su - ken
> getfacl /tmp/abc
user:ken:rwx
```

---
### **❓ How do you change the default login shell for all users in Linux?**

**Edit the shadow password suite configuration file `login.def`**
1. Run `vim /etc/login.defs`.
1. You can edit the username and password configuration here: 
```python
# Password aging controls:
#	PASS_MAX_DAYS	Maximum number of days a password may be used.
#	PASS_MIN_DAYS	Minimum number of days allowed between password changes.
#	PASS_MIN_LEN	Minimum acceptable password length.
#	PASS_WARN_AGE	Number of days warning given before a password expires.
PASS_MAX_DAYS	99999
PASS_MIN_DAYS	0
PASS_MIN_LEN	5
PASS_WARN_AGE	7
# Min/max values for automatic uid selection in useradd
UID_MIN                  1000
UID_MAX                 60000
# System accounts
SYS_UID_MIN               201
SYS_UID_MAX               999
# Min/max values for automatic gid selection in groupadd
GID_MIN                  1000
GID_MAX                 60000
# System accounts
SYS_GID_MIN               201
SYS_GID_MAX               999
```

**Edit the useradd configuration file**
1. Run `vim /etc/default/useradd` and change the `SHELL` path directory.
```python
# useradd defaults file
GROUP=100
HOME=/home
INACTIVE=-1
EXPIRE=
SHELL=/bin/sh # Edit here!
SKEL=/etc/skel
CREATE_MAIL_SPOOL=yes
```


---
### **❓After setting up passwordless ssh access, whenever I try to login to a server, it still asks me for a password. My public key is also set up in the remote server.**

1. Make sure the public key added to the remote server is given sufficient permissions using `chmod 400 myPublicKey.pem`
1. Then, run `ssh-add` to check if ssh agent forwarding is enabled and ssh connection is established.

---
### **❓ How do you disable the root user?**

1. Run `usermod -s /sbin/nologin root`
1. Then check your access as a root user using: 
```bash
> logout
> sudo su -
This account is currently not available.
```
1. To re-enable, run `sudo usermod -s /bin/bash root`

---
### **❓ A server is running the `httpd` service. How do you make sure `httpd` runs on only 2 CPU cores? (`CPUAffinity = 2`)**

1. Install `httpd` using `yum install httpd -y` for CentOS distributions. 
1. Install `mlocate` which is used to locate configuration files using `yum install mlocate -y`
1. Run `updatedb` to update the search indices of your local system database.
1. Run `locate httpd.service` to reveal the directory paths of all the `httpd` configuration files.
```bash
> locate httpd.service
/usr/lib/systemd/system/httpd.service
/usr/lib/systemd/system/httpd.service.d
/usr/share/man/man8/httpd.service.8.gz
```
1. Edit the `httpd` service configuration file using `/usr/lib/systemd/system/httpd.service`
1. Within the `httpd.service` config file, add `CPUAffinity=2` under `[Service]`
1. Restart the service using `systemctl daemon-reload && systemctl restart httpd` and check its status using `systemctl status httpd`



---
### **❓ After rebooting, there is a time difference between the `/var/log/message` and the OS time.**







1. There is a time difference between `date` and `cat /var/log/message/`.
1. There are 2 types of times in Linux:
    * Hardware time in BIOS - shown using `hwclock`
    * System time in OS - shown using `date`
1. The hardware time needs to synchronize with the system time. 
    * Run `hwclock --systohc` to set the hardware clock to the current system time OR
    * Run `hwclock --hctosys` to set the system time from the hardware clock.

---
### **❓ How to restart a service only if the service is already running? Stopped service should not be affected.**

1. Check if the service (e.g.: `httpd`) is running using `ps aux | grep httpd`
1. Restart the service using `systemctl restart httpd`. Make sure to run `systemctl daemon-reload` if you have modified the configuration file of the service before.
1. But this also restarts any stopped services. Hence, we use `systemctl try-restart httpd` instead to restart services that are only already running while stopped services are unaffected.

---
### **❓ What is the maximum filename length allowed in Linux?**

You can have a maximum of 255 characters in a filename in Linux

---
### **❓ How do you clear caches in Linux?**

1. To clear **pagecache**, run `echo 1 > /proc/sys/vm/drop_caches`
1. To clear **dentries** and **inodes**, run `echo 2 > /proc/sys/vm/drop_caches`
1. To clear all **pagecache**, **dentries** and **inodes**, run `echo 3 > /proc/sys/vm/drop_caches`. **Warning:** Not recommended in production systems as it will impact performance!